In [ ]:
from keras.models import load_model
import os
import transformers
from pathlib import Path
import torch
import numpy as np
from math import ceil
from random import shuffle
from itertools import chain
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Conv1D, MaxPooling1D, UpSampling1D
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as opt
import pandas as pd
from autoencoder import get_all_data
from sklearn import metrics


In [ ]:
autoencoder = load_model('lstm_model.h5')

In [ ]:
basePath = r'C:\Users\Himesh\Documents\thesis\extended'
positivePathSuffix = '/Positive'
negativePathSuffix = '/Negative'
tokenizerInPath = basePath + '\\tokenizerIn'
tokenizerOutPath = basePath + '\\tokenizerOut'
train_ratio = 0.7

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
smellList = ['ComplexMethod']
final_text = ""
print(tokenizerInPath)
for smell in smellList:
    smellPath = os.path.join(tokenizerInPath, smell,'Positive',"")
    #print(smellPath)
    
    for file in os.listdir(smellPath):
        #print(os.path.basename(file))
        with open(os.path.join(smellPath, file),"r") as read_file:
            try:
                text = read_file.read()
                tokenized_text = tokenizer.tokenize(text,padding = "max_length")
                input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
                modint = (len(input_ids)) % 512
                #print(modint)
                length = len(input_ids) - modint
            
                input_ids = input_ids[0:length]
                final_text += ' '.join(map(str, input_ids))+'\n'
            except Exception as e:
                print(e)
                pass
    #Path(os.path.join(tokenizerOutPath,smell,positivePathSuffix, 'tokenizer.tok')).touch(exist_ok=True)        
    with open(os.path.abspath(os.path.join(tokenizerOutPath,smell,'Positive', 'tokenizer.tok')),'w',errors='ignore') as out_file:
        #out_file.touch(exist_ok=True)
        #print(final_text)
        out_file.write(final_text)
    
    smellPath = os.path.join(tokenizerInPath, smell,'Negative',"")
print(smellPath)

for file in os.listdir(smellPath):
    #print(os.path.basename(file))
    with open(os.path.join(smellPath, file),"r") as read_file:
        try:
            text = read_file.read()
            tokenized_text = tokenizer.tokenize(text,padding = "max_length")
            input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
            modint = (len(input_ids)) % 512
            #print(modint)
            length = len(input_ids) - modint
           
            input_ids = input_ids[0:length]
           
            final_text += ' '.join(map(str, input_ids))+'\n'
        except Exception as e:
            print(e)
            
            pass
#Path(os.path.join(tokenizerOutPath,smell,positivePathSuffix, 'tokenizer.tok')).touch(exist_ok=True)        
with open(os.path.abspath(os.path.join(tokenizerOutPath,smell,'Negative', 'tokenizer.tok')),'w',errors='ignore') as out_file:
    #out_file.touch(exist_ok=True)
    #print(final_text)
    out_file.write(final_text)


In [ ]:
data_path = os.path.join(tokenizerOutPath, smell)
input_data = get_all_data(data_path, smell)
input_data.eval_data = input_data.eval_data.reshape((len(input_data.eval_labels), input_data.max_input_length, 1))
input_data.eval_data = input_data.eval_data[:,:894,:]
predictions = autoencoder.predict(input_data.eval_data)
predictions = predictions.reshape(predictions.shape[0], predictions.shape[1])
input_data.eval_data = input_data.eval_data.reshape(input_data.eval_data.shape[0], input_data.eval_data.shape[1])
mse = np.mean(np.power(input_data.eval_data - predictions, 2), axis=1)
error_df = pd.DataFrame({'Reconstruction_error': mse,
                            'True_class': input_data.eval_labels})
print(error_df.describe())


In [ ]:
print(predictions[1])
confusion_matrix = metrics.confusion_matrix(input_data.eval_labels, predictions)